In [1]:
import time
time.gmtime()

time.struct_time(tm_year=2021, tm_mon=9, tm_mday=12, tm_hour=11, tm_min=53, tm_sec=49, tm_wday=6, tm_yday=255, tm_isdst=0)

In [2]:
import os,sys,inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.insert(0,parentdir) 

In [3]:
import pandas as pd
import itertools, collections
import time
import numpy as np

from _code import DeepLogModel
from _code import generator
from _code import seed
from _code import trie

In [4]:
### Parameters
_seed=1
subdir = "static-1"
vocabulary_size = 20
num_patterns = 5

In [5]:
seed.seed(_seed)
subdir = subdir

In [6]:
vocabulary_size = vocabulary_size
num_patterns = num_patterns
vocabulary = generator.make_vocabulary(vocabulary_size=vocabulary_size)
patterns = generator.generate_patterns(num_patterns=num_patterns, vocabulary=vocabulary, min_pattern_size=3, max_pattern_size=7)
trie_g = trie.calc_g_value(patterns)
trie_h = trie.calc_h_value(patterns)
print("trie-g:", trie_g)
print("trie-h:", trie_h)
text_train, marks_train = generator.generate_text(patterns, text_size=50000, anomaly_ratio=0.00, vocabulary=vocabulary) 
tests = generator.generate_tests(patterns, vocabulary, n=1000, text_size = 4, anomaly_ratio=0.1)

trie-g: 5
trie-h: 2


In [7]:
import json, os

def write_inputs(subdir, vocabulary_size, vocabulary, patterns, text_train, marks_train, tests):
    dir_= os.path.join("..\\inputs", subdir)
    os.makedirs(dir_, exist_ok=True)
    j1={
        "vocabulary_size" : vocabulary_size,
        "vocabulary" : vocabulary,
        "workflows" : patterns
    }
    json.dump(j1, open(os.path.join(dir_, "v-wf.json"), "w"))

    j2 = {"text_train" : text_train, "marks_train" : marks_train}
    json.dump(j2, open(os.path.join(dir_, "train.json"), "w"))

    j3 = {"tests" : tests}
    json.dump(j3, open(os.path.join(dir_, "tests.json"), "w"))

def read_inputs(subdir):
    dir_ = os.path.join("..\\inputs", subdir)
    j1 = json.load(open(os.path.join(dir_, "v-wf.json")))
    j2 = json.load(open(os.path.join(dir_, "train.json")))
    j3 = json.load(open(os.path.join(dir_, "tests.json")))
    
    vocabulary_size = j1["vocabulary_size"]
    vocabulary = j1["vocabulary"]
    patterns = j1["workflows"]
    patterns = [tuple(p) for p in patterns]
    
    text_train = j2["text_train"]
    marks_train = j2["marks_train"]
    
    tests_str_keys = j3["tests"]
    tests = dict()
    for k,v in tests_str_keys.items():
        tests[int(k)] = tuple(v)
    
    
    return vocabulary_size, vocabulary, patterns, text_train, marks_train, tests

write_inputs(subdir, vocabulary_size, vocabulary, patterns, text_train, marks_train, tests)

In [8]:
ret_vocabulary_size, ret_vocabulary, ret_patterns, ret_text_train, ret_marks_train, ret_tests = read_inputs(subdir)

assert ret_vocabulary_size == vocabulary_size
assert ret_vocabulary == vocabulary
assert ret_patterns == patterns
assert ret_text_train == text_train
assert ret_marks_train == marks_train
assert ret_tests == tests


In [9]:
deep_log_model = DeepLogModel.DeepLogModel(h=trie_h+1, n=vocabulary_size, vocabulary=vocabulary)
deep_log_model.build(num_lstm_layers=2, lstm_size=64)
deep_log_model.fit(text_train,epochs=1)

In [10]:
b = time.time()
entries = list()
for k, (text_test, text_marks, anomaly) in tests.items():
    for g in range(0, vocabulary_size+1):
        res = deep_log_model.monitor_session(text_test, text_marks, g=g)
        entry = (k,g,res)
        entries.append(entry)
e = time.time()
print("time:", round(e-b,3), "seconds")

time: 1200.533 seconds


In [11]:
df = pd.DataFrame(entries, columns = ["i","g","status"])
ddf = df.pivot_table(index="i", columns = "g", values = "status", aggfunc = "sum")
def calc(s):
    c = dict(collections.Counter(s))
    TP = c.get("TP", 0)
    TN = c.get("TN", 0)
    FP = c.get("FP", 0)
    FN = c.get("FN", 0)
    eps = 1e-9
    
    prec = TP / (TP + FP + eps)
    rec = TP / (TP + FN + eps)
    acc = (TP + TN) / (TP + TN + FP + FN + eps)
    f1 = 2*(prec*rec)/(prec+rec+eps)
    
    return {"prec" : prec, "rec" : rec, "acc" : acc, "f1" : f1, "TP" : TP, "TN" : TN, "FP" : FP, "FN" : FN}
    
e=ddf.apply(calc, axis=0)
e1=pd.DataFrame(list(e.values))

print("measure results for each g-value")
e1

measure results for each g-value


,prec,rec,acc,f1,TP,TN,FP,FN
0,0.249000,1.000000,0.249,0.398719,249,0,751,0
1,0.251515,1.000000,0.259,0.401937,249,10,741,0
2,0.260188,1.000000,0.292,0.412935,249,43,708,0
3,0.296076,1.000000,0.408,0.456881,249,159,592,0
4,0.412752,0.987952,0.647,0.582249,246,401,350,3
5,1.000000,0.947791,0.987,0.973196,236,751,0,13
6,1.000000,0.891566,0.973,0.942675,222,751,0,27
7,1.000000,0.807229,0.952,0.893333,201,751,0,48
8,1.000000,0.771084,0.943,0.870748,192,751,0,57
9,1.000000,0.730924,0.933,0.844548,182,751,0,67


In [12]:

def write_outputs(subdir, df, ddf, e1):
    df.to_csv(os.path.join("..\inputs", subdir, "DL-results.csv"))
    ddf.to_csv(os.path.join("..\inputs", subdir, "DL-resuls_pivot.csv"))
    e1.to_csv(os.path.join("..\inputs", subdir, "DL-resutls_metrics.csv"))

write_outputs(subdir, df, ddf, e1)

In [13]:
display(e1.loc[[e1["acc"].argmax()]])
display(e1.loc[[e1["f1"].argmax()]])

,prec,rec,acc,f1,TP,TN,FP,FN
5,1.0,0.947791,0.987,0.973196,236,751,0,13


,prec,rec,acc,f1,TP,TN,FP,FN
5,1.0,0.947791,0.987,0.973196,236,751,0,13
